In [1]:
from datetime import datetime, timedelta
import gym
from gym.utils import seeding
import numpy as np
import random
import pandas as pd
from gym import spaces

In [51]:
from xml.dom.pulldom import parseString
import numpy as np
from StreetGraph import StreetGraph
import osmnx as ox
import networkx as nx
import folium
from folium.plugins import MarkerCluster
from datetime import datetime, timedelta
from array import array

class GraphEnv(gym.Env):

    MAX_STEPS = 10

    
    def __init__(self,env_config = None):
        env_config = env_config or {}
        """_summary_

        Args:
            graph (nx.MultiDiGraph): graph
            start_hub (int): nodeId
            final_hub (int): nodeId

        """  
        self.final_hub = 3
        self.start_hub = 6
        self.position = 6
        
        
        pickup_day = 1
        pickup_hour =  np.random.randint(24)
        pickup_minute = np.random.randint(60)
        self.pickup_time = datetime(2022,1,pickup_day,pickup_hour,pickup_minute,0)
        self.time = self.pickup_time
        self.total_travel_time = 0


        self.graph =StreetGraph('meinheim')
        self.graph.trips = StreetGraph('meinheim').trips


        
        self.seed()
        self.reset()

        # if self.graph.has_node(self.start_hub):
        #     self.position = self.start_hub
        # else:
        #     return 'Initialized start hub was not found in graph'

        # if self.graph.has_node(final_hub):
        #     self.final_hub = final_hub
        # else:
        #     return 'Initialized final hub was not found in graph'
        num_actions = len(self.availableActions())

        self.action_space = gym.spaces.Discrete(num_actions) 
        self.observation_space = gym.spaces.Discrete(len(list(self.graph.graph.nodes()))) #num of nodes in the graph
        

    def step(self, action: int):
        """ Executes an action based on the index passed as a parameter (only works with moves to direct neighbors as of now)

        Args:
            action (int): index of action to be taken from availableActions
        Returns:
            int: new position
            int: new reward
            boolean: isDone
        """
        self.count += 1

        old_position = list(self.graph.graph.nodes())[self.position]
        availableActions = self.availableActions()
        step_duration = 0

        if self.validateAction(action):
            if(action == 0):
                step_duration = 300
                pass
            else:
                selected_trip = availableActions[action-1]

                if( list(self.graph.graph.nodes())[self.final_hub] in selected_trip['route']):
                    route = selected_trip['route']

                    self.position = self.final_hub
                    index_in_route = route.index( list(self.graph.graph.nodes())[self.final_hub])
                    route_to_final_hub=route[:index_in_route]
                    print(route_to_final_hub)
                    print(route)
                    print(self.final_hub)
                    route_travel_time_to_final_hub = ox.utils_graph.get_route_edge_attributes(self.graph.graph,route_to_final_hub,attribute='travel_time')
                    step_duration = sum(route_travel_time_to_final_hub)

                else:
                    self.position = list(self.graph.graph.nodes()).index(selected_trip['target_node'])
                    route_travel_time = ox.utils_graph.get_route_edge_attributes(self.graph.graph,selected_trip['route'],attribute='travel_time')
                    step_duration = sum(route_travel_time)
                
                # Increase global time state by travelled time (does not include waiting yet, in this case it should be +xx seconds)
                self.time = selected_trip['departure_time']

                # Instead of cumulating trip duration here we return travel_time 
                # self.total_travel_time += timedelta(seconds=travel_time)

            self.time += timedelta(seconds=step_duration)
        else:
            pass

        

        return self.position, self.reward(),  self.isDone(), {}

    def availableActions(self):
        """ Returns the available actions at the current position. Uses a simplified action space with moves to all direct neighbors allowed.

        Returns:
            list: list of nodeIds of direct neighbors
        """
        rides = list(self.availableTrips())
        return rides

    def availableTrips(self):
        """ Returns a list of all available trips at the current node and within the next 5 minutes. Includes the time of departure from the current node as well as the target node of the trip.

        Returns:
            list: [departure_time,target_node]
        """
        position=list(self.graph.graph.nodes())[self.start_hub]
        position_str=str(list(self.graph.graph.nodes())[self.start_hub])
        start_timestamp=self.time
        time_window=5
        end_timestamp = self.time + timedelta(minutes=time_window)
        grid=self.graph.trips
        list_trips=[]
        paths=grid['node_timestamps']
        
        for index in range(len(paths)):
            dict = grid['node_timestamps'][index]
            for tupel_position in dict:
                position_timestamp= datetime.strptime(str(dict[tupel_position]), "%Y-%m-%d %H:%M:%S")
                inTimeframe = start_timestamp <= position_timestamp and end_timestamp >= position_timestamp
                startsInCurrentPosition = str(tupel_position) == position_str
                trip_target_node = grid['dropoff_node'][index]
                isNotFinalNode = str(tupel_position) != str(trip_target_node)
                
                if startsInCurrentPosition and inTimeframe and isNotFinalNode:
                    route = grid['route'][index]
                    index_in_route = route.index(position)
                    route_to_target_node=route[index_in_route::]
                    trip = {'departure_time': position_timestamp, 'target_node': trip_target_node, 'route': route_to_target_node}
                    list_trips.append(trip)
        return list_trips

    def validateAction(self, action):
        return action < len(self.availableTrips()) + 1

    def isDone(self):
        if (self.count >= self.MAX_STEPS | self.position == self.final_hub ):
            return True
        else:
            return False
    
    def reward(self): # TODO: extend function: should not return 0 reward if position is a second time on start_hub
        
        if self.isDone():
            return 10
        elif self.position == self.start_hub: 
            return 0
        else:
            return -1

    def render(self, visualize_actionspace: bool = False):
        """_summary_

        Args:
            visualize_actionspace (bool, optional): _description_. Defaults to False.

        Returns:
            _type_: _description_
        """
        current_pos_x = self.graph.nodes[list(self.graph.graph.nodes())[self.position]]['x']
        current_pos_y = self.graph.nodes[list(self.graph.graph.nodes())[self.position]]['y']
        final_hub_x = self.graph.nodes[list(self.graph.graph.nodes())[self.final_hub]]['x']
        final_hub_y = self.graph.nodes[list(self.graph.graph.nodes())[self.final_hub]]['y']
        start_hub_x = self.graph.nodes[list(self.graph.graph.nodes())[self.start_hub]]['x']
        start_hub_y = self.graph.nodes[list(self.graph.graph.nodes())[self.start_hub]]['y']

        # Create plot
        plot = ox.plot_graph_folium(self.graph,fit_bounds=True, weight=2, color="#333333")

        # Place markers for start, final and current position
        folium.Marker(location=[final_hub_y, final_hub_x], icon=folium.Icon(color='red', prefix='fa', icon='flag-checkered')).add_to(plot)
        folium.Marker(location=[start_hub_y, start_hub_x], popup = f"Pickup time: {self.pickup_time.strftime('%m/%d/%Y, %H:%M:%S')}", icon=folium.Icon(color='lightblue', prefix='fa', icon='caret-right')).add_to(plot)
        folium.Marker(location=[current_pos_y, current_pos_x], popup = f"Current time: {self.time.strftime('%m/%d/%Y, %H:%M:%S')}", icon=folium.Icon(color='lightgreen', prefix='fa',icon='cube')).add_to(plot)

        if(visualize_actionspace):
            for i, trip in enumerate(self.availableTrips()):
                target_node_x = self.graph.nodes[trip['target_node']]['x']
                target_node_y = self.graph.nodes[trip['target_node']]['y']
                popup = "%s: go to node %d" % (i, trip['target_node'])
                folium.Marker(location=[target_node_y, target_node_x], popup = popup, tooltip=str(i)).add_to(plot)

        # Plot
        pos_to_final = nx.shortest_path(self.graph, self.position, self.final_hub, weight="travel_time")
        if(not len(pos_to_final)< 2):
            ox.plot_route_folium(G=self.graph,route=pos_to_final,route_map=plot)

        return plot

    def reset(self):
        self.count = 0
        self.final_hub = 3
        self.start_hub = 6
        self.position = 6
        
        pickup_day = 1
        pickup_hour =  np.random.randint(24)
        pickup_minute = np.random.randint(60)
        self.pickup_time = datetime(2022,1,pickup_day,pickup_hour,pickup_minute,0)
        
        self.time = self.pickup_time
        self.total_travel_time = 0


        self.graph =StreetGraph('meinheim')
        self.graph.trips = StreetGraph('meinheim').trips


        return self.position
    


In [40]:

GRAPH_NAME = 'meinheim'
start_hub = 290333444
final_hub = 280430536
pickup_time=datetime(2022,1,1,1,1,0)

streetgraph = StreetGraph(GRAPH_NAME)

#env = Environment(streetgraph, start_hub, final_hub, pickup_time)


In [46]:
#without ppo trainer
def run_one_episode (env):
    env.reset()
    sum_reward = 0
    for i in range(10):
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print(sum_reward)
            break
    return sum_reward

env=GraphEnv()
for i in range(1):
    run_one_episode (env)

-1


In [3]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym

import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

In [4]:
ray.init()

2022-03-26 09:05:47,837	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2022-03-26_09-05-45_858297_49231/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-26_09-05-45_858297_49231/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-03-26_09-05-45_858297_49231',
 'metrics_export_port': 61472,
 'gcs_address': '127.0.0.1:56159',
 'node_id': '40a6ea0674f281283088bdb49620dd2fdd9362ab393a7379f4641c2e'}

In [5]:
trainer_config = DEFAULT_CONFIG.copy()
trainer_config['num_workers'] = 1
trainer_config["train_batch_size"] = 400
trainer_config["sgd_minibatch_size"] = 64
trainer_config["num_sgd_iter"] = 10
trainer_config["framework"] = "torch"

In [52]:
trainer = PPOTrainer(trainer_config,GraphEnv )

(RolloutWorker pid=50742) 2022-03-26 10:29:37,986	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-03-26 10:29:38,062	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [ ]:
checkpoint_root = "tmp/ppo/gridworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

In [55]:
results = []
episode_data = []
episode_json = []

for n in range(1):
    result = trainer.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

StopIteration: 